In [11]:
import numpy as np
from src.kinematics_library.guassian import Gaussian


mu_x = np.array([[1.0], [2.0], [0.0]])          # Initial guess
P = np.diag([1.0, 1.0, 0.2])                   # Uncertainty
prior = Gaussian.from_moment(mu_x, P)


In [ ]:
from src.kinematics_library.gaussian_measurement_function import MeasurementFunctionProtocol
from src.kinematics_library.gaussian_return import GaussianReturn
import numpy as np

class DummyTransform(MeasurementFunctionProtocol):
    def __init__(self):
        super().__init__()
        self.beacon_1 = np.array([[2.0], [0.0], [1.0]])  # (3,1)

    def __call__(self, x: np.ndarray, return_grad: bool = False, return_hess: bool = False) -> GaussianReturn:
        d = self.beacon_1 - x  # (3,1)
        norm_d = np.linalg.norm(d)  # scalar

        y = np.array([[norm_d]])  # (1,1) output measurement

        grad = None
        hess = None

        if return_grad:
            grad = (-d / norm_d).T  # (1,3) Jacobian (row vector)
        
        if return_hess:
            # TODO: second derivative (Hessian) if you want (optional for now)
            pass

        return GaussianReturn(
            magnitude=y,
            grad_magnitude=grad,
            hess_magnitude=hess
        )

rv = DummyTransform()(x=mu_x,return_grad=True)
rv

GaussianReturn(magnitude=array([[2.44948974]]), gaussian_magnitude=None, grad_magnitude=array([[-0.40824829,  0.81649658, -0.40824829]]), hess_magnitude=None)

In [ ]:

p = DummyTransform()
h = p.__call__
noise = None

mu = prior.mu
S = prior.sqrt_cov

result: GaussianReturn = h(x=mu, return_grad=True)

if not result.has_grad:
    raise ValueError("Affine transform requires gradient information (Jacobian) when no Gaussian is provided")

y_part = result.magnitude
J = result.grad_magnitude

if y_part.ndim == 1:
    y_part = y_part[:, None]

# Propagate uncertainty
SJ_T = J @ S.T
parts = [SJ_T]

if noise is not None:
    parts.append(noise.sqrt_cov)

# Stack all uncertainty contributions vertically
max_cols = max(p.shape[1] for p in parts)
padded_parts = [
    np.pad(p, ((0, 0), (0, max_cols - p.shape[1]))) if p.shape[1] < max_cols else p
    for p in parts
]
stacked = np.vstack(padded_parts)

# QR decomposition to obtain square-root of covariance
_, R = np.linalg.qr(stacked, mode="reduced")
SR = R.T

# Ensure full (n, n) shape
expected_dim = J.shape[0]
curr_rows, curr_cols = SR.shape
if curr_rows < expected_dim:
    pad_rows = expected_dim - curr_rows
    SR = np.pad(SR, ((0, pad_rows), (0, 0)))
if curr_cols < expected_dim:
    pad_cols = expected_dim - curr_cols
    SR = np.pad(SR, ((0, 0), (0, pad_cols)))

rF = Gaussian(y_part, SR)

In [14]:
y_part

array([[2.44948974]])

In [15]:
rF.mu

array([[2.44948974]])

We have a measurement that is as expected. Now we're going to get a gaussian that's [h(x); x] to prep for the conditional

In [ ]:
return_hessian = False
return_gradient = False
system = None

def predict_density(x, system, return_gradient=False, return_hessian=False) -> GaussianReturn:
    return DummyTransform()(x=x,return_grad=return_gradient)

x = prior.mu

if return_hessian:
    py, dhdx, d2hdx2 = predict_density(x, system, return_gradient=True, return_hessian=True)

elif return_gradient:
    py, dhdx = predict_density(x, system, return_gradient=True)
    d2hdx2 = np.zeros((py.mu.shape[0], x.shape[0], x.shape[0]))
else:
    rv = predict_density(x, system)
    py = rv.gaussian_magnitude
    dhdx = np.zeros((py.mu.shape[0], x.shape[0]))
    d2hdx2 = np.zeros((py.mu.shape[0], x.shape[0], x.shape[0]))

ny = py.mu.shape[0]
nx = x.shape[0]

# Construct full S_aug: (ny+nx, ny+nx)
S_aug = np.zeros((ny + nx, ny + nx))
S_aug[:ny, :ny] = py.sqrt_cov  # top-left block is R
S_aug[ny:, ny:] = system.density.sqrt_cov  # bottom-right = state uncertainty


mu_aug = np.vstack([py.mu, x])
J_aug = np.vstack([dhdx, np.eye(nx)])

py_aug = Gaussian(mu_aug, S_aug)

return py_aug, J_aug

In [ ]:
# P_y = H P_x H^T + R
# = [1 3] [1 0; 0 4] [1;3] + 0.5
# = (1*1*1 + 1*0*3 + 3*0*1 + 3*4*3) + 0.5
# = (1 + 0 + 0 + 36) + 0.5 = 37 + 0.5 = 37.5
    